In [5]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

42


In [6]:
# (p. 236) Linear regression with TensorFlow on the housing data set
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
# standard normal equation using TF API
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [7]:
# (p. 237) Gradient descent with TF
n_epochs = 1000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
# this is one learning step:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 8.13765
Epoch 100 MSE = 4.934
Epoch 200 MSE = 4.8816
Epoch 300 MSE = 4.86176
Epoch 400 MSE = 4.84768
Epoch 500 MSE = 4.83717
Epoch 600 MSE = 4.82929
Epoch 700 MSE = 4.82334
Epoch 800 MSE = 4.81884
Epoch 900 MSE = 4.81538
Best theta: [[  8.79584312e-01  -5.57167292e-01   9.56155300e-01  -2.08472013e-01
   -1.99861288e-01   1.87136412e-01   5.65419912e-01  -7.57135391e-01]
 [  8.31322670e-01   8.66321564e-01   9.19650793e-01   9.13227618e-01
    8.96506071e-01   8.59153211e-01   8.28257561e-01   9.17337656e-01]
 [  1.46341637e-01   1.44680440e-01   1.55231565e-01   1.51387796e-01
    1.36577368e-01   1.73406914e-01   1.42296046e-01   1.35773748e-01]
 [ -2.17739627e-01  -2.99242944e-01  -3.98968130e-01  -3.91769975e-01
   -3.82013083e-01  -2.29457706e-01  -2.18439713e-01  -4.30179894e-01]
 [  2.43749514e-01   3.17907989e-01   3.99918556e-01   3.96137923e-01
    3.97672117e-01   2.35468477e-01   2.47182906e-01   4.41375971e-01]
 [  5.51197864e-03   4.45242040e-03   7.59896915

In [8]:
# (p. 238) same, but use TF for determining the gradients to make more efficient
n_epochs = 1000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


# Here's the difference to the previous cell:
gradients = tf.gradients(mse, [theta])[0]

# this is one learning step:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 8.74963
Epoch 100 MSE = 6.85482
Epoch 200 MSE = 5.96279
Epoch 300 MSE = 5.51776
Epoch 400 MSE = 5.28225
Epoch 500 MSE = 5.15045
Epoch 600 MSE = 5.07273
Epoch 700 MSE = 5.02464
Epoch 800 MSE = 4.99344
Epoch 900 MSE = 4.97224
Best theta: [[-0.08415127  0.78658867  0.87453079  0.77808094  0.0769608  -0.11199379
   0.86463547 -0.14286542]
 [ 0.93849033  0.5504958   0.3828384   0.90956193  0.70648617  0.6419583
   0.67956781  0.67263067]
 [ 0.20393579  0.10857365  0.04017963  0.16313778  0.25188375  0.19615449
   0.22836216  0.13829674]
 [-0.53218174  0.40696776  0.68423885 -0.35147581  0.05803951  0.1733541
   0.08970544  0.10141062]
 [ 0.48968521 -0.430489   -0.63592947  0.30668247 -0.01198027 -0.15823807
  -0.06529286 -0.09215629]
 [ 0.02789194 -0.09039758 -0.06870309  0.01017972  0.08040939  0.0593757
   0.07408393  0.00836333]
 [-0.07658326 -0.05952287 -0.01234639 -0.05016567 -0.011805   -0.09659304
  -0.09973767 -0.00376804]
 [ 0.12515043  0.07192492 -0.54701722  0.09111

In [10]:
# (p. 239) same, but use TF optimizer
n_epochs = 1000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


# Again, here's the difference to the previous cell:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# Alternative optimizer:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 8.32642
Epoch 100 MSE = 4.92854
Epoch 200 MSE = 4.88466
Epoch 300 MSE = 4.85884
Epoch 400 MSE = 4.84142
Epoch 500 MSE = 4.82964
Epoch 600 MSE = 4.82163
Epoch 700 MSE = 4.81614
Epoch 800 MSE = 4.81237
Epoch 900 MSE = 4.80974
Best theta: [[  5.83661318e-01  -5.40048838e-01  -2.68336058e-01   4.16929245e-01
   -4.15213585e-01  -3.00775290e-01  -1.72068834e-01  -7.65746832e-01]
 [  8.81991208e-01   8.54847729e-01   8.52063119e-01   8.34859848e-01
    8.08563232e-01   9.08813834e-01   8.47350538e-01   8.14230502e-01]
 [  1.56922460e-01   1.31041974e-01   1.41281098e-01   1.38656601e-01
    1.38677016e-01   1.35050163e-01   1.39863402e-01   1.38646364e-01]
 [ -3.11446995e-01  -2.99052060e-01  -2.73663372e-01  -2.40036160e-01
   -1.81093067e-01  -4.12440062e-01  -2.65758812e-01  -1.93845049e-01]
 [  3.20347786e-01   3.27200085e-01   2.97354341e-01   2.69053131e-01
    2.16239989e-01   4.25987870e-01   2.91257203e-01   2.27684021e-01]
 [  8.70319456e-03  -3.59434780e-04   3.40155

In [12]:
# (p. 240) Batch feeding and TensorBoard
n_epochs = 1000
learning_rate = 0.01

import numpy.random as rnd

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    rnd.seed(epoch * n_batches + batch_index)
    indices = rnd.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch


theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


# Again, here's the difference to the previous cell:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# Alternative optimizer:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})    
            best_theta = theta.eval()
        print("Epoch", epoch, "MSE =", mse.eval())
    print("Best theta:", best_theta)


InvalidArgumentError: Shape [-1,9] has negative dimensions
	 [[Node: X_9 = Placeholder[dtype=DT_FLOAT, shape=[?,9], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'X_9', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ax/code/handson-ml/env/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ax/code/handson-ml/env/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ax/code/handson-ml/env/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/ax/code/handson-ml/env/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/ax/code/handson-ml/env/l